# 线性回归的简洁实现

In [28]:
import numpy as np
import torch
from torch.utils import data

In [29]:
def synthetic_data(w, b, num_examle):
    """
    生成y=wx+b+噪声
    """
    x = torch.normal(0, 1, (num_examle, len(w)))
    y = torch.matmul(x, w) + b
    y += torch.normal(0, 0.01, y.shape)    # 添加均值为0，方差为0.01的噪音
    return x, y.reshape((-1, 1))

In [30]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [31]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.0418,  0.6353],
         [-0.7187,  0.6377],
         [-0.1373, -0.5707],
         [-1.3882,  0.2628],
         [ 0.0223, -0.1671],
         [ 0.0247, -0.6380],
         [-1.0788, -0.6416],
         [ 0.0267, -0.1015],
         [-0.9624, -0.7738],
         [ 1.5560,  0.8802]]),
 tensor([[2.1139],
         [0.5894],
         [5.8460],
         [0.5273],
         [4.7997],
         [6.4165],
         [4.2396],
         [4.5958],
         [4.9187],
         [4.3234]])]

## 使用框架的预定义好的层

In [32]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))    # 2为输入维度， 1为输出维度
# Sequential 相当于list of layers 一个容器, 将一个线性层放入一个容器中

初始化模型参数

In [33]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用MSELoss类，也称为L2范数

In [34]:
# 计算均方误差使用MSELoss类，也称为L2范数
loss = nn.MSELoss()

实例化SGD实例

In [35]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# paramters是参数， lr是学习率

训练模型

In [36]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000251
epoch 2, loss 0.000105
epoch 3, loss 0.000104
